In [2]:
import pandas as pd

In [3]:
df = pd.read_pickle('../data/processed/forecasts_pmafe.pkl')

- Complexity: number of firms for which analyst i supplied at least one forecast during the first 11 months of year t OR number of two-digit SICs for which analyst i supplied at least one forecast during the first 11 months of year t minus the average number of two-digit SICs followed by an analyst following firm j at time t.

In [5]:
number_of_forecasts_by_analyst_pro_period = df.groupby(['fiscal_period_ending', 'analyst'])['official_ticker'].count().reset_index()\
    .sort_values(by=['fiscal_period_ending', 'official_ticker'], ascending=[True, False]).set_index(['fiscal_period_ending', 'analyst'])

In [17]:
number_of_forecasts_by_analyst_pro_period

official_ticker
fiscal_period_ending analyst                 
2015-01-31           106026                14
                     118217                11
                     127622                11
                     108441                10
                     114475                10
...                                       ...
2023-07-31           196694                 1
                     197705                 1
                     198190                 1
                     198735                 1
                     199274                 1

[79245 rows x 1 columns]

- Forecasting ability/experience: number of years for which analyst i supplied at least one forecast during the first 11 months of the year through year t

In [51]:
old_data = pd.read_csv('../data/1990-2014.zip')
old_data.head()

,TICKER,OFTIC,CNAME,ACTDATS,ANALYS,FPI,MEASURE,FPEDATS,ANNDATS
0,0000,TLMR,TALMER BANCORP,2014-03-11,119962,1,EPS,2014-12-31,2014-03-09
1,0000,TLMR,TALMER BANCORP,2014-03-11,80474,1,EPS,2014-12-31,2014-03-10
2,0000,TLMR,TALMER BANCORP,2014-03-11,50789,1,EPS,2014-12-31,2014-03-10
3,0000,TLMR,TALMER BANCORP,2014-03-11,79092,1,EPS,2014-12-31,2014-03-10
4,0000,TLMR,TALMER BANCORP,2014-03-17,71182,1,EPS,2014-12-31,2014-03-17


In [52]:
analysts_experience = old_data[['ANALYS', 'ANNDATS']].drop_duplicates().reset_index(drop=True)
analysts_experience

,ANALYS,ANNDATS
0,119962,2014-03-09
1,80474,2014-03-10
2,50789,2014-03-10
3,79092,2014-03-10
4,71182,2014-03-17
...,...,...
2138012,136140,2013-11-10
2138013,41097,2013-11-14
2138014,136140,2014-09-28
2138015,41097,2014-10-13


In [53]:
analysts_experience['ANNDATS'] = pd.to_datetime(analysts_experience['ANNDATS'])
analysts_experience.dtypes 

ANALYS              int64
ANNDATS    datetime64[ns]
dtype: object

In [55]:
analysts_experience['YEAR'] = analysts_experience['ANNDATS'].dt.year
analysts_experience.head()

,ANALYS,ANNDATS,YEAR
0,119962,2014-03-09,2014
1,80474,2014-03-10,2014
2,50789,2014-03-10,2014
3,79092,2014-03-10,2014
4,71182,2014-03-17,2014


In [56]:
analysts_experience = analysts_experience.drop(columns=['ANNDATS'])
analysts_experience = analysts_experience.drop_duplicates()
sorted_analysts = analysts_experience.groupby('ANALYS').count()
sorted_analysts = sorted_analysts.sort_values(by=['YEAR'], ascending=False)
sorted_analysts